In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Dropout, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform
from matplotlib.pyplot import imshow
import cv2 as ocv
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data(dir, class_names, num_of_img_per_class, input_shape):
    classes = class_names
    num_of_img_per_class = num_of_img_per_class

    data = []
    labels = []

    class_counter = 2
    for category in os.listdir(dir):

        if class_counter==2:
            print("Loading Data:", end=" ")
        new_dir = os.path.join(dir,category)

        for img in os.listdir(new_dir):
            if len(data)<(num_of_img_per_class * class_counter):
                img_path = os.path.join(new_dir,img)
                if (np.all(np.array(ocv.imread(img_path,0)))==0):
                    data.append(ocv.resize(ocv.imread(img_path,1),(input_shape[0],input_shape[1])))
                    labels.append(classes.index(category))
                    if len(data)/(num_of_img_per_class*2)*100%20==0:
                        print(int(len(data)/(num_of_img_per_class*2)*100),"%", end="  ")

        class_counter=class_counter+1

        if class_counter==3:
            print(". Complete.", end=" ")

    combined = list(zip(data,labels))
    random.shuffle(combined)
    data, labels = zip(*combined)
    Data = np.array(data)/255
    Labels = np.array(labels)
    n_col = Data.shape[1]
    n_rows = Labels.shape[0]
    return (Data, Labels)


def identity_block(X, f, filters, training=True, initializer=random_uniform):
    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Dropout(0.3)(X) #
    X = Activation('relu')(X)

    #Second component of main path
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Dropout(0.3)(X) #
    X = Activation('relu')(X)

    #Third component of main path
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)

    #Adding shortcut value to main path, and pass it through a RELU activation
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)

    return X


def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X

    #First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Dropout(0.3)(X) #
    X = Activation('relu')(X)

    #Second component of main path
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Dropout(0.3)(X) #
    X = Activation('relu')(X)

    #Third component of main path
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)

    #SHORTCUT PATH
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)

    #Adding shortcut value to main path and passing it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


def ResNet50(input_shape = (64, 64, 3), classes = 2):
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    #Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    #Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256]) #

    #Stage 3
    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512]) #

    #Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    #Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    #AVGPOOL
    X = AveragePooling2D((2, 2))(X)

    #Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)

    #Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
dir = '/content/drive/MyDrive/Colab Notebooks/의료 프로젝트/hist/Cardiomegaly/train'
classes = ['false','true']
num_of_img_per_class = 2000
input_shape = (128, 128, 3)

x_train, y_train = load_data(dir, classes, num_of_img_per_class, input_shape)

Loading Data: 20 %  40 %  . Complete. 60 %  80 %  

In [ ]:
model = ResNet50(input_shape, classes = 2)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_split=0.20, epochs = 30, batch_size = 32)

In [ ]:
# 모델 저장
model.save('./Cardiomegaly_model.h5')

In [ ]:
import os
import csv

# 이미지 파일이 들어 있는 폴더 경로
folder_path = '/content/drive/MyDrive/Colab Notebooks/의료 프로젝트/Cardiomegaly/train/train/true'

# CSV 파일 경로와 이름
csv_file = '/content/drive/MyDrive/Colab Notebooks/의료 프로젝트/Cardiomegaly/train_data1.csv'

# CSV 파일 헤더
header = ['name', 'labels']

# 이미지 파일 확장자
image_extensions = ['.png']

# 폴더 내 모든 파일 목록 조회
file_list = os.listdir(folder_path)

# 이미지 파일과 라벨을 저장할 리스트 초기화
data = []

# 폴더 내 파일 순회
for file_name in file_list:
    # 파일의 확장자 확인
    ext = os.path.splitext(file_name)[1].lower()
    if ext in image_extensions:
        # 이미지 파일이라면
        label = 1
        #file_name = file_name.replace(' ','')
        #file_name = file_name.replace('','')
        data.append([file_name, label])

# CSV 파일 생성 및 데이터 쓰기
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data)

print('CSV 파일이 생성되었습니다.')

CSV 파일이 생성되었습니다.


In [ ]:
import pandas as pd

train_img_path = '/content/drive/MyDrive/Cardiomegaly/train/train/total1'
labels = pd.read_excel(r'/content/drive/MyDrive/Cardiomegaly/train_data.xlsx')

In [ ]:
#Extracting different classes
classes = sorted(labels['target'].unique())
n_classes = len(classes)
print(f'number of class: {n_classes}')

number of class: 2


In [ ]:
classes_to_num = dict(zip(classes,range(n_classes)))

In [ ]:
#Function to load and convert images to array
from keras.utils import load_img
from keras.utils import to_categorical

def images_to_array(data_dir,df,image_size):
    image_names = df['file_name']
    image_labels = df['target']
    data_size = len(image_names)

    X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
    y = np.zeros([data_size,1],dtype = np.uint8)

    for i in range(data_size):
        img_name = image_names[i]
        img_dir = os.path.join(data_dir,img_name)
        img_pixels = load_img(img_dir,target_size=image_size)
        X[i] = img_pixels
        y[i] = classes_to_num[image_labels[i]]

    y = to_categorical(y)
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y

In [ ]:
#Selecting image size according to pretrained models
img_size = (299,299,3) # 128, 128, 3
X, y = images_to_array(train_img_path,labels,img_size)

Ouptut Data Size:  (4438, 299, 299, 3)
Ouptut Label Size:  (4438, 2)


In [ ]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Dropout(0.3)(X) #
    X = Activation('relu')(X)

    #Second component of main path
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Dropout(0.3)(X) #
    X = Activation('relu')(X)

    #Third component of main path
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)

    #Adding shortcut value to main path, and pass it through a RELU activation
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)

    return X


def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X

    #First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Dropout(0.3)(X) #
    X = Activation('relu')(X)

    #Second component of main path
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Dropout(0.3)(X) #
    X = Activation('relu')(X)

    #Third component of main path
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)

    #SHORTCUT PATH
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)

    #Adding shortcut value to main path and passing it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


def ResNet50(input_shape = (64, 64, 3), classes = 2):
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    #Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    #Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256]) #
    X = identity_block(X, 3, [64, 64, 256]) #

    #Stage 3
    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512]) #

    #Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    #Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    #AVGPOOL
    X = AveragePooling2D((2, 2))(X)

    #Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)

    #Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
input_shape = (299,299, 3)

In [ ]:
model = ResNet50(input_shape, classes = 2)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, validation_split=0.20, epochs = 30, batch_size = 32)

Epoch 1/30
111/111 [==============================] - 149s 944ms/step - loss: 2.1192 - accuracy: 0.4972 - val_loss: 0.9591 - val_accuracy: 0.5090
Epoch 2/30
111/111 [==============================] - 108s 971ms/step - loss: 0.8710 - accuracy: 0.5214 - val_loss: 0.7470 - val_accuracy: 0.6115
Epoch 3/30
111/111 [==============================] - 108s 972ms/step - loss: 0.7291 - accuracy: 0.6065 - val_loss: 0.6793 - val_accuracy: 0.6329
Epoch 4/30
111/111 [==============================] - 104s 939ms/step - loss: 0.7316 - accuracy: 0.6003 - val_loss: 0.6557 - val_accuracy: 0.6565
Epoch 5/30
111/111 [==============================] - 108s 972ms/step - loss: 0.6774 - accuracy: 0.6465 - val_loss: 0.6489 - val_accuracy: 0.6779
Epoch 6/30
111/111 [==============================] - 108s 972ms/step - loss: 0.6589 - accuracy: 0.6513 - val_loss: 0.6649 - val_accuracy: 0.6982
Epoch 7/30
111/111 [==============================] - 108s 972ms/step - loss: 0.6762 - accuracy: 0.6549 - val_loss: 0.6174 -

In [ ]:
# 모델 저장
model.save('./Cardiomegaly_model.h5')